# Appendix  - Source Code

### Import Libraries

In [ ]:
# Import Libraries
import warnings
import pymongo as pm
from datetime import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pprint
import json
import random
import time
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tsa.stattools import adfuller
from matplotlib.colors import LinearSegmentedColormap

# Ignoring Warnings
warnings.simplefilter('ignore', ConvergenceWarning)

# Set Seaborn Style for All Plots
sns.set_style("whitegrid")

### Accessing the Database

In [ ]:
# Connecting to Database Using Credentials

db_add = "<Your DB Address>"
auth_source = "<Auth Source>"
username = "<Your Username>"
password = "<Your Password>"


client = pm.MongoClient(db_add,
        ssl=True,
        authSource = auth_source,
        tlsAllowInvalidCertificates=True,
        username=username,
        password=password)

db = client['carsharing']

# Get Permanent Bookings Collections

c2g_booking_perm = db['PermanentBookings']
enj_booking_perm = db['enjoy_PermanentBookings']

### Step 3 – Lab #2 - Prediction using ARIMA models

### S3_T1

In [ ]:
# Query to get rentals per hour
def get_cars_per_Hour_bookFilter(city, Unix_startTime, Unix_endTime):
    
    query = [
        {
            '$match': { 'city': city,
                    'init_time': {'$gte': Unix_startTime, '$lte': Unix_endTime}
                    }
        },
        {
            '$project' : { '_id':0,
                        'city': 1,
                        'hourOfDay': {'$dateFromParts': {'year':{'$year':'$init_date'}, 'month': {'$month':'$init_date'}, 'day': {
                        '$dayOfMonth': '$init_date'}, 'hour': {'$hour': '$init_date'}}},
                        'duration':{'$ceil': {'$subtract': ['$final_time',
                        '$init_time']} },
                        'moved': {'$ne': [
                    {'$arrayElemAt': ['$origin_destination.coordinates', 0]},
                    {'$arrayElemAt': ['$origin_destination.coordinates', 1]}]
                    }
                    }
        },
        {
            '$match': {
                    'moved': True,
                    'duration': {'$gte': 4*60, '$lte': 2*60*60}
                    }
        },
        {
            '$group': { '_id': '$hourOfDay',
                    'count':{'$sum':1}
        }
        },
        {
            '$sort': {'_id': 1}
        }
    ]
    return query

In [ ]:
# For each city, run the query and fetch the data accordingly
cities=['Seattle', 'Milano Enjoy', 'Milano']

results = {}
for city in cities:
    city_dict = {'bookings':{}}
    startingDate = dt(2017, 10, 1, 0) # Init Time
    endingDate = dt(2017, 11, 1, 0) # End Time    
    Unix_startTime = dt.timestamp(startingDate)
    Unix_endTime = dt.timestamp(endingDate)
    # Make Ready the Query for the collection
    if 'Enjoy' in city:
        book_pipe = get_cars_per_Hour_bookFilter('Milano', Unix_startTime, Unix_endTime)
        i_booking = enj_booking_perm
    else:
        book_pipe = get_cars_per_Hour_bookFilter(city, Unix_startTime, Unix_endTime)
        i_booking = c2g_booking_perm
        
    # Assign proper collection for the aggregation
    daily_bookings = i_booking.aggregate(book_pipe)
    
    for item in daily_bookings:
        city_dict['bookings'][item['_id']] = item['count']
        
    # Dictionary to DataFrame Convertion
    city_df_from_dict = pd.DataFrame.from_dict(city_dict)
    city_df = city_df_from_dict.reindex(pd.date_range(start="2017-10-01",end="2017-11-01",freq='H'),fill_value=0)
    results[city] = city_df[:-1]                 

### S3_T2

In [ ]:
# Handling Missing Data
# If there were any missings: replace with last or next week of data
all_dates = []
missing_dates = {}
week_list = pd.date_range(start="2017-10-01",end="2017-11-01",freq='H').strftime("%Y-%m-%d %H:%M:%S").tolist()
for day in week_list: all_dates.append(dt.strptime(day, '%Y-%m-%d  %H:%M:%S'))
for city in cities:
    missing_dates[city] = []
    for date in all_dates[:-1]:
        if results[city].loc[date]['bookings']==0:
            missing_dates[city].append(date)
            if date.day>=25 or date.month==11:
                results[city].loc[date]['bookings'] = results[city].loc[date.replace(day=date.day-7)]['bookings'] + random.randint(0,10)
            else:
                results[city].loc[date]['bookings'] = results[city].loc[date.replace(day=date.day+7)]['bookings'] + random.randint(0,10)
        if city=='Milano Enjoy':
            if results[city].loc[date]['bookings']>=500:
                results[city].loc[date]['bookings'] = (results[city].loc[date+timedelta(days=1)]['bookings']+results[city].loc[date-timedelta(days=-1)]['bookings'])/2

### S3_T3

In [ ]:
# Plot Missing Values and Rolling Statistics

# Adjusting plots rcParams for the proper fontsizes

plt.rc('axes', labelsize=30)
plt.rc('ytick', labelsize=34)
plt.rc('xtick', labelsize=34)    
plt.rc('legend', fontsize=28)
plt.rc('figure', titlesize=28)
fig, axes = plt.subplots(1, 3, figsize=(50, 10), tight_layout=True)
date_ticks = np.arange("2017-10-01","2017-11-01", dtype ='datetime64')
fig.subplots_adjust(wspace=0.08)

#Get X and Y Axis for Each City
x_seattle, y_seattle = zip(*sorted(results['Seattle']['bookings'].items()))
x_milano, y_milano = zip(*sorted(results['Milano']['bookings'].items()))
x_milano_enj, y_milano_enj = zip(*sorted(results['Milano Enjoy']['bookings'].items()))

# Rolling Statistics for 168h (7d) for Seattle Car2Go
r_mean_seattle, r_std_seattle = results['Seattle']['bookings'].rolling(168).mean(), results['Seattle'].rolling(168).std()
axes[0].tick_params(axis='x', labelrotation = 45)
axes[0].set_title("Rolling Statistics and Missing Values for Seattle (Car2Go)", fontsize=36)
axes[0].set_xlabel("Time [h]", fontsize=36)
axes[0].set_ylabel("No. Rentals", fontsize=36)
axes[0].vlines(missing_dates['Seattle'], color="red", linestyle="--", alpha=1, ymin=0, ymax=600, label=f"{len(missing_dates['Seattle'])} Missing Values") 
axes[0].plot(r_mean_seattle, label='Rolling Mean', linewidth=4, color='k')
axes[0].plot(r_std_seattle, label='Rolling Std', linewidth=4, color='c')
sns.lineplot(data=results['Seattle'], x=x_seattle, y=y_seattle, ax=axes[0], color='g')

# Rolling Statistics for 168h (7d) for Milano Car2Go
r_mean_milano, r_std_milano = results['Milano']['bookings'].rolling(168).mean(), results['Milano']['bookings'].rolling(168).std()
axes[1].tick_params(axis='x', labelrotation = 45)
axes[1].set_title("Rolling Statistics and Missing Values for Milano (Car2Go)", fontsize=36)
axes[1].set_xlabel("Time [h]", fontsize=36)
axes[1].vlines(missing_dates['Milano'], color="red", linestyle="--", alpha=1, ymin=0, ymax=600, label=f"{len(missing_dates['Milano'])} Missing Values") 
axes[1].plot(r_mean_milano, label='Rolling Mean', linewidth=4, color='k')
axes[1].plot(r_std_milano, label='Rolling Std', linewidth=4, color='c')
sns.lineplot(data=results['Milano'], x=x_milano, y=y_milano, ax=axes[1], color='g')

# Rolling Statistics for 168h (7d) for Milano Enjoy
r_mean_milano_enj, r_std_milano_enj = results['Milano Enjoy']['bookings'].rolling(168).mean(), results['Milano Enjoy']['bookings'].rolling(168).std()
axes[2].tick_params(axis='x', labelrotation = 45)
axes[2].set_title("Rolling Statistics and Missing Values for Milano (Enjoy)", fontsize=36)
axes[2].set_xlabel("Time [h]", fontsize=36)
axes[2].vlines(missing_dates['Milano Enjoy'], color="red", linestyle="--", alpha=1, ymin=0, ymax=600, label=f"{len(missing_dates['Milano Enjoy'])} Missing Values") 
axes[2].plot(r_mean_milano_enj, label='Rolling Mean', linewidth=4, color='k')
axes[2].plot(r_std_milano_enj, label='Rolling Std', linewidth=4, color='c')
sns.lineplot(data=results['Milano Enjoy'], x=x_milano_enj, y=y_milano_enj, ax=axes[2], color='g')
plt.savefig('S3_T3.png', dpi=300)
plt.show()

### S3_T4

In [ ]:
# Adjusting plots rcParams for the proper fontsizes
plt.rc('axes', labelsize=2)
plt.rc('ytick', labelsize=10)
plt.rc('xtick', labelsize=10)    
plt.rc('legend', fontsize=14)
plt.rc('figure', titlesize=22)
plt.rcParams.update({'axes.titlesize': 'medium'})

fig, axes = plt.subplots(2, 3, figsize=(12, 5), tight_layout=True)
fig.tight_layout()

# Statsmodel ACF and PACF plots
sm.graphics.tsa.plot_acf(results['Seattle']['bookings'].values, lags=48, title='ACF - Seattle (Car2Go)', ax=axes[0,0], color='k', vlines_kwargs={"colors": 'blueviolet'})
sm.graphics.tsa.plot_pacf(results['Seattle']['bookings'].values, lags=48, title='PACF - Seattle (Car2Go) ', ax=axes[1,0], color='k', vlines_kwargs={"colors": 'blueviolet'})
sm.graphics.tsa.plot_acf(results['Milano']['bookings'].values, lags=48, title='ACF - Milano (Car2Go)', ax=axes[0,1], color='k', vlines_kwargs={"colors": 'blueviolet'})
sm.graphics.tsa.plot_pacf(results['Milano']['bookings'].values, lags=48, title='PACF - Milano (Car2Go)', ax=axes[1,1], color='k', vlines_kwargs={"colors": 'blueviolet'})
sm.graphics.tsa.plot_acf(results['Milano Enjoy']['bookings'].values, lags=48, title='ACF - Milano (Enjoy)', ax=axes[0,2], color='k', vlines_kwargs={"colors": 'blueviolet'})
sm.graphics.tsa.plot_pacf(results['Milano Enjoy']['bookings'].values, lags=48, title='PACF - Milano (Enjoy)', ax=axes[1,2], color='k', vlines_kwargs={"colors": 'blueviolet'})
fig.subplots_adjust(wspace=0.2, hspace=0.3)
plt.savefig('S3_T4.png', dpi=300)
plt.show()

# Adfuller Stationarity check
print(f"Augmented Dickey Fuller (ADF) Test for Seattle (Car2Go): {adfuller(results['Seattle'].values)[1]} - < 0.005. Good! It's Stationary!")
print(f"Augmented Dickey Fuller (ADF) Test for Milano (Car2Go): {adfuller(results['Milano'].values)[1]} - < 0.005. Good! It's Stationary!")
print(f"Augmented Dickey Fuller (ADF) Test for Milano (Enjoy): {adfuller(results['Milano Enjoy'].values)[1]} - < 0.005. Good! It's Stationary!")

### S3_T6

In [ ]:
# General ARIMA fit function
def ARIMA_fit(data, city, p, q, train_size, test_size, mode="expanding"):
    
    # Train, test split
    counter = 0
    train_list = []
    test_list = []
    for index in data.index.values:
        if counter <= train_size:
            train_list.append(data.loc[index]['bookings'])
            counter += 1
        elif counter > train_size and counter <= test_size+train_size:
            test_list.append(data.loc[index]['bookings'])
            counter += 1
        else:
            break
    
    # Stepwise fit and prediction for test data
    predictions = []
    for row in range(test_size):
        model = ARIMA(train_list, order=(p, 0, q))
        model = model.fit(method='statespace')
        pred = model.forecast()
        predictions.append(pred[0])
        observation = test_list[row]
        if mode=="expanding":
            train_list.append(observation)
        elif mode=="sliding":
            train_list.append(observation)
            train_list.pop(0) # Drop the first value for sliding
        
    return test_list, predictions

In [ ]:
# ARMA max hyperparameters and train test split ratio
days_to_train = 21
days_to_test = 7
p_values = {'Seattle':3, 'Milano':2, 'Milano Enjoy':3}
q_values = {'Seattle':4, 'Milano':4, 'Milano Enjoy':4}

In [ ]:
# Loop over cities for the maximum possible value for p and q in expanding mode
ARIMA_results = {}
tic = time.perf_counter()
for city in cities:
    print(city)
    ARIMA_results[city] = ARIMA_fit(results[city], city, p=p_values[city], q=q_values[city],
                                    train_size=days_to_train*24, test_size=days_to_test*24,
                                    mode="expanding")
toc = time.perf_counter()
print(f"Job Finished in {toc-tic} seconds.")

In [ ]:
# Function to call all the errors associated with the lab
def get_errors(original, predicitions):
    MSE = mean_squared_error(original, predicitions)
    MAE = mean_absolute_error(original, predicitions)
    MAPE = mean_absolute_percentage_error(original, predicitions)
    R2 = r2_score(original, predicitions)
    return [MSE, MAE, MAPE, R2]

In [ ]:
# Adjusting plots rcParams for the proper fontsizes

plt.rc('axes', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('xtick', labelsize=20)    
plt.rc('legend', fontsize=14)
plt.rc('figure', titlesize=18)
plt.rcParams.update({'axes.titlesize': 'small'})
fig, axes = plt.subplots(1, 3, figsize=(22, 5), tight_layout=True)

fig.tight_layout()

# Get element-wise error for better inspection
# Plot real, predicted, and error
error = np.abs(np.subtract(ARIMA_results['Seattle'][0], ARIMA_results['Seattle'][1]))
axes[0].plot(ARIMA_results['Seattle'][0], color='black',label='Real Rentals', linewidth=3) 
axes[0].plot(ARIMA_results['Seattle'][1], color='limegreen', label='Prediction', linewidth=2)
axes[0].bar(np.arange(0,len(ARIMA_results['Seattle'][0]),1), error, color='orangered', label='Error', width=1)
axes[0].set_title(f"Seattle (Car2Go): ARIMA ({p_values['Seattle']},0,{q_values['Seattle']}) with Expanding Window", fontsize=18)
axes[0].legend(loc='best')
axes[0].set_xlabel('Test [h]')
axes[0].set_ylabel('No. Rentals')
axes[0].set_ylim(0, 250)

error = np.abs(np.subtract(ARIMA_results['Milano'][0], ARIMA_results['Milano'][1]))
axes[1].plot(ARIMA_results['Milano'][0], color='black', label='Real Rentals', linewidth=3) 
axes[1].plot(ARIMA_results['Milano'][1], color='limegreen', label='Prediction', linewidth=2)
axes[1].bar(np.arange(0,len(ARIMA_results['Milano'][0]),1), error, color='orangered', label='Error', width=1)
axes[1].set_title(f"Milano (Car2Go): ARIMA ({p_values['Milano']},0,{q_values['Milano']}) with Expanding Window", fontsize=18)
axes[1].legend(loc='best')
axes[1].set_xlabel('Test [h]')
axes[1].set_ylim(0, 500)

error = np.abs(np.subtract(ARIMA_results['Milano Enjoy'][0], ARIMA_results['Milano Enjoy'][1]))
axes[2].plot(ARIMA_results['Milano Enjoy'][0], color='black', label='Real Rentals', linewidth=3) 
axes[2].plot(ARIMA_results['Milano Enjoy'][1], color='limegreen', label='Prediction', linewidth=2)
axes[2].bar(np.arange(0,len(ARIMA_results['Milano Enjoy'][0]),1), error, color='orangered', label='Error', width=1)
axes[2].set_title(f"Milano (Enjoy): ARIMA ({p_values['Milano Enjoy']},0,{q_values['Milano Enjoy']}) with Expanding Window", fontsize=18)
axes[2].legend(loc='best')
axes[2].set_xlabel('Test [h]')
axes[2].set_ylim(0, 500)
plt.savefig('S3_T6.png', dpi=300)
plt.show()

# Table for total errors
error_results = [
    get_errors(ARIMA_results['Seattle'][0], ARIMA_results['Seattle'][1]),
    get_errors(ARIMA_results['Milano'][0], ARIMA_results['Milano'][1]),
    get_errors(ARIMA_results['Milano Enjoy'][0], ARIMA_results['Milano Enjoy'][1])]

print(pd.DataFrame(error_results, index=["Seattle (Car2Go)", "Milano (Car2Go)", "Milano (Enjoy)"],
                   columns=["MSE", "MAE", "MAPE", "R2"]))

### S3_T7.A

In [ ]:
# List of all inputs for P and Q GridSearch
input_list = []
for city in cities:
    for p in range(0, p_values[city]+1):
        for q in range(0, q_values[city]+1):
            if p==0 and q==0:
                continue
            input_list.append([results[city], city, p, q])

In [ ]:
# Create a process function to run in paraller
# This function only works in other IDEs (not notebook)
def ARIMA_Thread(input_list):
    data, city, p, q = input_list
    days_to_train = 21
    days_to_test = 7
    original, predictions = ARIMA_fit(data, city, p=p, q=q, train_size=days_to_train*24, test_size=days_to_test*24, mode="expanding")
    MSE, MAE, MAPE, R2 = get_errors(original, predictions)
    return [original, predictions, city, p, q, MSE, MAE, MAPE, R2]

In [ ]:
# This cell can be easily implemented using multiprocessing
# Loop over input list and get the required data
counter = 0
ARIMA_GridSearch = []
tic1 = time.perf_counter()
for inp in input_list:
    counter += 1
    tic2 = time.perf_counter()
    print(f"Input {counter}: City={inp[1]}-P={inp[2]}-Q={inp[3]}")
    ARIMA_GridSearch.append(ARIMA_Thread(inp))
    toc2 = time.perf_counter()
    print(f"Iteration Finished in {toc2-tic2} seconds.")
toc1 = time.perf_counter()
print(f"Job Finished in {toc1-tic1} seconds.")

headers = ['City', 'p', 'q', 'MSE', 'MAE', 'MAPE', 'R2', 'Original', 'Prediction']
ARIMA_GridSearch_Results = pd.DataFrame(columns=headers)

# Organize results in DataFrame
for elem in ARIMA_GridSearch:
    ARIMA_GridSearch_Results.loc[len(ARIMA_GridSearch_Results)] = [elem[2], elem[3], elem[4], elem[5], elem[6], elem[7], elem[8], elem[0], elem[1]]

In [ ]:
# Uncomment if you want to save/read the grid search data

#ARIMA_GridSearch_Results.to_csv('GridSearchV2.csv')
#ARIMA_GS = pd.read_csv('GridSearchV2.csv')

ARIMA_GS = ARIMA_GridSearch_Results.copy()

In [ ]:
# Plot a single ARMA for different P values predictions, q=2

ARIMA_GS_ALLP = ARIMA_GS.drop(columns=['Unnamed: 0'])
ARIMA_All_P = ARIMA_GS_ALLP[(ARIMA_GS_ALLP['q']==2)].drop(columns=['q', 'MSE', 'MAE', 'MAPE', 'R2'])

plt.rc('axes', labelsize=24)
plt.rc('ytick', labelsize=22)
plt.rc('xtick', labelsize=22)    
plt.rc('legend', fontsize=18)
plt.rc('figure', titlesize=18)
plt.rcParams.update({'axes.titlesize': 'small'})

plt.figure(figsize=(22, 6), tight_layout=True)
axes = plt.plot(ARIMA_results['Milano Enjoy'][0], color='black', label='Real Bookings', linewidth=4) 
for p in range(0, p_values['Milano Enjoy']+1):
    axes = plt.plot(json.loads(ARIMA_All_P[(ARIMA_All_P['City']=='Milano Enjoy')&(ARIMA_All_P['p']==p)]['Prediction'].iloc[0]), label=f'P = {p}', linewidth=2)
plt.title(f"Milano (Enjoy): ARIMA with Expanding Window", fontsize=26)
plt.legend(loc='upper left')
plt.xlabel('Test [h]')
plt.ylabel('No. Rentals')
plt.xlim(0, 175)
plt.ylim(0, 700)
plt.savefig('S3_T6_MilanoEnjoy_AllP.png', dpi=300)
plt.show()

In [ ]:
# Preparing a dictionary for MAPE heatmap plot
GS_dict = {}
for city in cities:
    GS_MAPE = pd.DataFrame(index=[x for x in range(0, p_values[city]+1)], columns=[x for x in range(0, q_values[city]+1)])
    for p in range(0, p_values[city]+1):
        for q in range(0, q_values[city]+1):
            temp = ARIMA_GS[ARIMA_GS['City']==city].drop(columns=['City', 'MSE', 'MAE', 'R2', 'Original', 'Prediction'])
            if p==0 and q==0:
                GS_MAPE.iloc[p,q] = np.nan # P=0, Q=0? No way!
            else:
                GS_MAPE.iloc[p,q] = round(list(temp.loc[(temp['p']==int(p)) & (temp['q']==int(q)), 'MAPE'])[0],4)
    GS_dict[city] = pd.DataFrame(GS_MAPE.values.tolist(),
                   columns = GS_MAPE.columns.tolist(), index =GS_MAPE.index.tolist())

In [ ]:
# Adjusting plots rcParams for the proper fontsizes

plt.rc('axes', labelsize=22)
plt.rc('ytick', labelsize=20)
plt.rc('xtick', labelsize=20)    
plt.rc('legend', fontsize=10)
plt.rc('figure', titlesize=18)
plt.rcParams.update({'axes.titlesize': 'xx-large'})
plt.figure(tight_layout=True)

cmap = LinearSegmentedColormap.from_list(
    name='ARIMA_GS', 
    colors=['green','white','red']
)

for city in cities:
    plt.figure(figsize=(9, 5))
    data_matrix = GS_dict[city].to_numpy()
    min_in_each_column = np.sort(data_matrix.flatten())[2:3]
    axes = sns.heatmap(data_matrix,
                linewidth=0.8,
                annot=True,
                cmap=cmap,
                fmt=".2%",
                vmin=0,
                vmax=1,
                annot_kws={"size": 22, 'color':'k'})
    # Mask for Bolding the 2 minimum values
    axes = sns.heatmap(data_matrix,
                mask=data_matrix >= min_in_each_column,
                annot_kws={"weight": "bold", "size": 22, 'color':'k'},
                linewidth=0.8,
                annot=True,
                cbar=False,
                cmap=cmap,
                fmt=".2%",
                vmin=0,
                vmax=1)

    axes.set(xlabel='Q for MA', ylabel='P for AR')
    if 'Enjoy' in city:
        title = f'MAPE for Milano (Enjoy)'
    else:
        title = f'MAPE for {city} (Car2Go)'
    axes.set_title(title, fontsize=24)
    axes.invert_yaxis()
    plt.tight_layout()
    plt.savefig(f"S3_T7_A_{city}.png", dpi=300)
    plt.show()

In [ ]:
# The best p and q values based on the previous heatmap results
# Run ARMA for the best p and q values

days_to_train = 21
p_values_best = {'Seattle':2, 'Milano':2, 'Milano Enjoy':2}
q_values_best = {'Seattle':2, 'Milano':1, 'Milano Enjoy':1}

ARIMA_results = {}
tic = time.perf_counter()
for city in cities:
    print(city)
    ARIMA_results[city] = ARIMA_fit(results[city], city, p=p_values_best[city], q=q_values_best[city],
                                    train_size=days_to_train*24, test_size=days_to_test*24,
                                    mode="expanding")
toc = time.perf_counter()
print(f"Job Finished in {toc-tic} seconds.")

In [ ]:
# Adjusting plots rcParams for the proper fontsizes
# Plot the ARMA for best p and q values

plt.rc('axes', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('xtick', labelsize=20)    
plt.rc('legend', fontsize=14)
plt.rc('figure', titlesize=18)
plt.rcParams.update({'axes.titlesize': 'small'})
fig, axes = plt.subplots(1, 3, figsize=(22, 5), tight_layout=True)

fig.tight_layout()

error = np.abs(np.subtract(ARIMA_results['Seattle'][0], ARIMA_results['Seattle'][1]))
axes[0].plot(ARIMA_results['Seattle'][0], color='black',label='Real Bookings', linewidth=3) 
axes[0].plot(ARIMA_results['Seattle'][1], color='limegreen', label='Prediction', linewidth=2)
axes[0].bar(np.arange(0,len(ARIMA_results['Seattle'][0]),1), error, color='orangered', label='Error', width=1)
axes[0].set_title(f"Seattle (Car2Go): ARIMA ({p_values_best['Seattle']},0,{q_values_best['Seattle']}) with Expanding Window", fontsize=18)
axes[0].legend(loc='best')
axes[0].set_xlabel('Test [h]')
axes[0].set_ylabel('No. Rentals')
axes[0].set_ylim(0, 250)

error = np.abs(np.subtract(ARIMA_results['Milano'][0], ARIMA_results['Milano'][1]))
axes[1].plot(ARIMA_results['Milano'][0], color='black', label='Real Bookings', linewidth=3) 
axes[1].plot(ARIMA_results['Milano'][1], color='limegreen', label='Prediction', linewidth=2)
axes[1].bar(np.arange(0,len(ARIMA_results['Milano'][0]),1), error, color='orangered', label='Error', width=1)
axes[1].set_title(f"Milano (Car2Go): ARIMA ({p_values_best['Milano']},0,{q_values_best['Milano']}) with Expanding Window", fontsize=18)
axes[1].legend(loc='best')
axes[1].set_xlabel('Test [h]')
axes[1].set_ylim(0, 500)

error = np.abs(np.subtract(ARIMA_results['Milano Enjoy'][0], ARIMA_results['Milano Enjoy'][1]))
axes[2].plot(ARIMA_results['Milano Enjoy'][0], color='black', label='Real Bookings', linewidth=3) 
axes[2].plot(ARIMA_results['Milano Enjoy'][1], color='limegreen', label='Prediction', linewidth=2)
axes[2].bar(np.arange(0,len(ARIMA_results['Milano Enjoy'][0]),1), error, color='orangered', label='Error', width=1)
axes[2].set_title(f"Milano (Enjoy): ARIMA ({p_values_best['Milano Enjoy']},0,{q_values_best['Milano Enjoy']}) with Expanding Window", fontsize=18)
axes[2].legend(loc='best')
axes[2].set_xlabel('Test [h]')
axes[2].set_ylim(0, 500)
plt.savefig('S3_T7_ARIMAFinal.png', dpi=300)
plt.show()

error_results = [
    get_errors(ARIMA_results['Seattle'][0], ARIMA_results['Seattle'][1]),
    get_errors(ARIMA_results['Milano'][0], ARIMA_results['Milano'][1]),
    get_errors(ARIMA_results['Milano Enjoy'][0], ARIMA_results['Milano Enjoy'][1])]

print(pd.DataFrame(error_results, index=["Seattle (Car2Go)", "Milano (Car2Go)", "Milano (Enjoy)"],
                   columns=["MSE", "MAE", "MAPE", "R2"]))

### S3_T7.B&C

In [ ]:
# Find the Best N (Past Samples) and the learning strategy
# Expanding window or Sliding window
days_to_train = np.arange(7,22,2)
days_to_test = 7
modes = ["expanding", "sliding"]

counter = 0
ARIMA_NSearch = []
tic1 = time.perf_counter()
for city in cities:
    for mode in modes:
        for period in days_to_train:
            counter += 1
            tic2 = time.perf_counter()
            print(f"Input {counter}: Period={period}-City={city}-P={p_values_best[city]}-Q={q_values_best[city]}-Mode={mode}")
            original, predictions = ARIMA_fit(results[city], city, p=p_values_best[city], q=q_values_best[city],
                                            train_size=period*24, test_size=days_to_test*24,
                                            mode=mode)
            [MSE, MAE, MAPE, R2] = get_errors(original, predictions)
            ARIMA_NSearch.append([city, period, mode, MSE, MAE, MAPE, R2, original, predictions])
            toc2 = time.perf_counter()
            print(f"Iteration Finished in {toc2-tic2} seconds.")
toc1 = time.perf_counter()
print(f"Job Finished in {toc1-tic1} seconds.")

In [ ]:
headers = ['City', 'N', 'Mode', 'MSE', 'MAE', 'MAPE', 'R2', 'Original', 'Prediction']
ARIMA_NSearch_Results = pd.DataFrame(columns=headers)
for elem in ARIMA_NSearch:
    ARIMA_NSearch_Results.loc[len(ARIMA_NSearch_Results)] = elem
#ARIMA_NSearch_Results.to_csv('NSearchV2.csv')

In [ ]:
ARIMA_GNS = ARIMA_NSearch_Results.drop(columns=['MSE', 'MAE', 'R2', 'Original', 'Prediction'])

In [ ]:
GNS_dict = {}
for city in cities:
    GNS_dict[city] = ARIMA_GNS[ARIMA_GNS['City']==city]

In [ ]:
# Adjusting plots rcParams for the proper fontsizes
# Plot model with different N and Learning strategies

plt.rc('axes', labelsize=28)
plt.rc('ytick', labelsize=24)
plt.rc('xtick', labelsize=24)    
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=26)
plt.rcParams.update({'axes.titlesize': 'small'})

plt.figure(figsize=(10, 8), tight_layout=True)
temp_city = GNS_dict['Seattle']
#Expanding Line Plot
N_plot_seattle = sns.lineplot(x=temp_city[temp_city['Mode']=='expanding']['N'],
                              y=temp_city[temp_city['Mode']=='expanding']['MAPE']*100,
                             data=temp_city, marker="o", label='Expanding Window Strategy',
                                linewidth=4, markersize=10, color='blueviolet')
#Sliding Line Plot
N_plot_seattle = sns.lineplot(x=temp_city[temp_city['Mode']=='sliding']['N'],
                              y=temp_city[temp_city['Mode']=='sliding']['MAPE']*100,
                             data=temp_city, marker="o", label='Sliding Window Strategy',
                                linewidth=4, markersize=10, color='orangered')

N_plot_seattle.set_xlabel('No. Past Samples')
N_plot_seattle.set_ylabel('MAPE %')
plt.tight_layout()
plt.xticks(np.arange(7,22,2))
plt.yticks(np.arange(15,50,5))
plt.legend()
plt.title('MAPE for Seattle (Car2Go)', fontsize=32)
plt.savefig(f"S3_T7_BC_Seattle.png", dpi=300)
plt.show()

plt.figure(figsize=(10, 8), tight_layout=True)
temp_city = GNS_dict['Milano']

#Expanding Line Plot
N_plot_Milano = sns.lineplot(x=temp_city[temp_city['Mode']=='expanding']['N'],
                              y=temp_city[temp_city['Mode']=='expanding']['MAPE']*100,
                             data=temp_city, marker="o", label='Expanding Window Strategy',
                                linewidth=4, markersize=10, color='blueviolet')
#Sliding Line Plot
N_plot_Milano = sns.lineplot(x=temp_city[temp_city['Mode']=='sliding']['N'],
                              y=temp_city[temp_city['Mode']=='sliding']['MAPE']*100,
                             data=temp_city, marker="o", label='Sliding Window Strategy',
                                linewidth=4, markersize=10, color='orangered')
N_plot_Milano.set_xlabel('No. Past Samples')
N_plot_Milano.set_ylabel('MAPE %')
plt.tight_layout()
plt.xticks(np.arange(7,22,2))
plt.yticks(np.arange(15,50,5))
plt.legend()
plt.title('MAPE for Milano (Car2Go)', fontsize=32)
plt.savefig(f"S3_T7_BC_Milano.png", dpi=300)
plt.show()

plt.figure(figsize=(10, 8), tight_layout=True)
temp_city = GNS_dict['Milano Enjoy']
#Expanding Line Plot
N_plot_Milano_enj = sns.lineplot(x=temp_city[temp_city['Mode']=='expanding']['N'],
                              y=temp_city[temp_city['Mode']=='expanding']['MAPE']*100,
                             data=temp_city, marker="o", label='Expanding Window Strategy',
                                linewidth=4, markersize=10, color='blueviolet')
#Sliding Line Plot
N_plot_Milano_enj = sns.lineplot(x=temp_city[temp_city['Mode']=='sliding']['N'],
                              y=temp_city[temp_city['Mode']=='sliding']['MAPE']*100,
                             data=temp_city, marker="o", label='Sliding Window Strategy',
                                linewidth=4, markersize=10, color='orangered')

N_plot_Milano_enj.set_xlabel('No. Past Samples')
N_plot_Milano_enj.set_ylabel('MAPE %')
plt.tight_layout()
plt.xticks(np.arange(7,22,2))
plt.yticks(np.arange(15,50,5))
plt.legend()
plt.title('MAPE for Milano (Enjoy)', fontsize=32)
plt.savefig(f"S3_T7_BC_MilanoEnjoy.png", dpi=300)
plt.show()

### S3_T8 [OPTIONAL]

In [ ]:
# Look for different horizon values

counter = 0
train_list = []
test_list = []

days_to_train = {'Seattle':21, 'Milano':21, 'Milano Enjoy':11}
days_to_test = 7
mode_to_train = {'Seattle':'expanding', 'Milano':'expanding', 'Milano Enjoy':'expanding'}
horizon_vals = range(1, 24)

all_preds = np.zeros((len(horizon_vals), days_to_test*24))
all_preds.fill(np.nan)
predictions = {'Seattle':all_preds.copy(), 'Milano':all_preds.copy(), 'Milano Enjoy':all_preds.copy()}
MAPE = np.zeros((len(horizon_vals)))
all_MAPE = {'Seattle':MAPE.copy(), 'Milano':MAPE.copy(), 'Milano Enjoy':MAPE.copy()}

tic1 = time.perf_counter()
for city in cities:
    train_size, test_size = days_to_train[city]*24, days_to_test*24
    for horizon in horizon_vals:
        print(f"ARIMA for City: {city} - Horizon: {horizon}")
        tic2 = time.perf_counter()
        for index in results[city].index.values:
            if counter <= train_size:
                train_list.append(results[city].loc[index]['bookings'])
                counter += 1
            elif counter > train_size and counter <= test_size+train_size:
                test_list.append(results[city].loc[index]['bookings'])
                counter += 1
            else:
                break
        train_list = [x for x in train_list]
        test_list = [x for x in test_list]
        
        for row in range(test_size):
            model = ARIMA(train_list, order=(p_values_best[city], 0, q_values_best[city]))
            model = model.fit(method='statespace')
            pred = model.forecast(steps=horizon)
            if row+horizon > test_size:
                break
            predictions[city][horizon-1][row+horizon-1] = pred[-1]
            observation = test_list[row]
            if mode_to_train[city]=="expanding":
                train_list.append(observation)
            elif mode_to_train[city]=="sliding":
                train_list.extend(observation)
                train_list.pop(0)
        all_MAPE[city][horizon-1] = mean_absolute_percentage_error(test_list[horizon-1:], predictions[city][horizon-1][horizon-1:])
        toc2 = time.perf_counter()
        print(f"Iteration Finished in {toc2-tic2} seconds.")
tic1 = time.perf_counter()
print(f"Job Finished in {toc1-tic1} seconds.")

In [ ]:
with open("all_MAPE.txt", 'w') as f: 
    for key, value in all_MAPE.items(): 
        f.write('%s:%s\n' % (key, value))
        
with open("preds.txt", 'w') as f: 
    for key, value in predictions.items(): 
        f.write('%s:%s\n' % (key, value))

In [ ]:
plt.rc('axes', labelsize=28)
plt.rc('ytick', labelsize=24)
plt.rc('xtick', labelsize=24)    
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=26)
plt.rcParams.update({'axes.titlesize': 'small'})

plt.figure(figsize=(10, 8), tight_layout=True)

for city in cities:
    plt.plot(horizon_vals, RES_MAPE[city], linewidth=2.5, label=f"{city}", marker='o')
plt.xlim([0,25])
plt.yticks(np.arange(0,2.05,0.25))
plt.legend(['Seattle (Car2Go)', 'Milano (Enjoy)', 'Milano (Car2Go)'])
plt.xlabel('Horizon [h]')
plt.ylabel('MAPE %')
plt.title('ARMA with Expanding Window', fontsize=22)
plt.savefig('S3_T8.png', dpi=300)
plt.show()

In [ ]:
plt.rc('axes', labelsize=28)
plt.rc('ytick', labelsize=24)
plt.rc('xtick', labelsize=24)    
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=26)
plt.rcParams.update({'axes.titlesize': 'small'})

plt.figure(figsize=(20, 8), tight_layout=True)


plt.plot(ARIMA_results['Seattle'][0], linewidth=4, label=f"Real Rentals", color='k')
for horizon in range(1,24,3):
    plt.plot(RES_PRED['Seattle'][horizon-1], linewidth=1.5, label=f"h = {horizon}")
plt.xlabel('Test [h]')
plt.ylabel('No. Rentals')
plt.title('Prediction for Different Horizons', fontsize=22)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.17),
          ncol=9, fancybox=True, shadow=True)
plt.savefig('S3_T8_2.png', dpi=300)
plt.show()